In [61]:
from torchvision import models
import torch.nn as nn

In [62]:
model = models.resnet18(pretrained=True)

In [63]:
print(model.parameters)

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (rel

### If we want layer1 and layer2 output, re-construct the network as below, there are 2 ways to achieve this
1.  Construct two Sequential
2.  Concatenate two layers as long list, then give the list to one Sequential 

In [64]:
# Method 1
original_model = models.resnet18(pretrained=True)

class ResLayer1Layer2(nn.Module):
            def __init__(self):
                super(ResLayer1Layer2, self).__init__()
                
                self.layer1 = nn.Sequential(*list(original_model.layer1.children()))
                self.layer2 = nn.Sequential(*list(original_model.layer2.children()))
                
            def forward(self, x):
                x = self.layer1(x)
                x = self.layer2(x)
                return x

new_model = ResLayer1Layer2()

In [65]:
new_model.parameters
#As Can be seen below, there are two Sequentials

<bound method Module.parameters of ResLayer1Layer2(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d

In [66]:
# Method 2
original_model = models.resnet18(pretrained=True)

class ResLayer1Layer2(nn.Module):
            def __init__(self):
                super(ResLayer1Layer2, self).__init__()
                list_of_layer = list(original_model.layer1.children())
                list_of_layer.extend(list(original_model.layer2.children()))
                self.layers = nn.Sequential(*list_of_layer)
                
            def forward(self, x):
                x = self.layers(x)
                return x

new_model = ResLayer1Layer2()

In [67]:
new_model.parameters

<bound method Module.parameters of ResLayer1Layer2(
  (layers): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3)

Now we construct a new resnet18, it contains all layers except the last (avgpool) and (fc) layer

The output of this network is direct Conv output

In [74]:
import torch
import torch.nn as nn
from torchvision import models



class ResnetLayer4(nn.Module):
            def __init__(self):
                super(ResnetLayer4, self).__init__()
                self.conv1 = ori_model.conv1
                self.bn1 = ori_model.bn1
                self.relu = ori_model.relu
                self.maxpool = ori_model.maxpool
                self.layer1 = nn.Sequential(*list(ori_model.layer1.children()))
                self.layer2 = nn.Sequential(*list(ori_model.layer2.children()))
                self.layer3 = nn.Sequential(*list(ori_model.layer3.children()))
                self.layer4 = nn.Sequential(*list(ori_model.layer4.children()))
                self.maxpool1 = nn.MaxPool2d(kernel_size=7, stride=1, padding=0)
            def forward(self, x):
                x = self.conv1(x)
                x = self.bn1(x)
                x = self.relu(x)
                x = self.maxpool(x)
                
                x = self.layer1(x)
                x = self.layer2(x)
                x = self.layer3(x)
                x = self.layer4(x)
                x = self.maxpool1(x)
                return x

if __name__ == '__main__':
    ori_model = models.resnet18(pretrained=True)
    new_model = ResnetLayer4()

In [75]:
new_model.parameters

<bound method Module.parameters of ResnetLayer4(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [52]:
inputs = torch.randn(1,3,224,224)

In [53]:
out = new_model(inputs)

In [54]:
out.size()

torch.Size([1, 512, 1, 1])

In [55]:
out = out.squeeze(0)
out = out.squeeze(1)
out = out.squeeze(1)

In [56]:
print(out)

tensor([4.6496, 4.2051, 4.8751, 6.0405, 5.3268, 3.6048, 4.4454, 5.6617, 5.2194,
        5.5882, 4.9849, 4.3703, 4.1351, 3.4922, 3.7416, 4.7014, 3.2803, 6.7772,
        5.5570, 4.7118, 4.0375, 4.3011, 4.9946, 4.5869, 4.0414, 5.2050, 4.6571,
        4.6277, 5.4674, 5.1948, 3.7852, 6.0094, 6.1718, 5.2869, 4.2914, 5.3220,
        5.4145, 4.7790, 4.3334, 3.9105, 5.3287, 5.5735, 4.4467, 3.6999, 3.5890,
        4.5310, 3.6460, 7.1843, 4.9932, 6.3305, 3.6334, 5.0162, 5.0512, 4.5096,
        4.5309, 6.3289, 4.8399, 4.6202, 3.1994, 4.1838, 4.2217, 4.3356, 6.3169,
        4.0796, 3.8922, 3.8285, 3.1958, 4.3715, 6.5252, 3.9025, 4.2065, 5.2079,
        4.6404, 5.6284, 5.3857, 4.4606, 4.3131, 4.8897, 3.7051, 5.7929, 5.0704,
        3.8280, 5.3356, 5.7185, 4.5074, 3.4898, 5.1851, 3.9330, 5.4083, 4.1672,
        4.8127, 4.3308, 4.6644, 4.7786, 3.5845, 4.0048, 5.1221, 5.0678, 4.8391,
        4.0416, 4.7152, 3.9937, 5.8160, 5.3591, 4.8777, 5.5091, 4.6134, 4.9775,
        4.5020, 4.0424, 3.8392, 3.6794, 